# Settings

## Imports

In [1]:
import sys,tweepy,csv,re, requests, json
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import os.path, time, re


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

# Getting Twitter

## API

In [4]:
#token
config = dotenv_values(".env")
bearer_token = config['BEARER_TOKEN']

#connections
auth = tweepy.OAuth2BearerHandler({bearer_token})
api = tweepy.API(auth)



def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def get_tweet_v1(query, filename, max_n):
    search_url = 'https://api.twitter.com/2/tweets/search/recent'


    file_name = f'{filename}.bz2'
    
    if os.path.exists(file_name) == False: #First checking if database exists
        print(f'Getting tweets...')
        
        # Querying the API
        json_response = connect_to_endpoint(search_url, query)
        
        tweets_dt = pd.DataFrame.from_dict(json_response['data'])
        
        try:
            n_token = json_response['meta']["next_token"]
            n = 0
            while n_token != 0 | n < max_n:
                print(f'Next Token: {n} \n {n_token}')
                query_next = query
                query_next['next_token'] = n_token
                json_response = connect_to_endpoint(search_url, query_next)
                tweets_n = pd.DataFrame.from_dict(json_response['data'])
                tweets_dt = pd.concat([tweets_dt,tweets_n], ignore_index=True)

                n += 1
                n_token = json_response['meta']["next_token"]

        except:
            print('Error to proceed')
            
        meta = json_response['meta']
        np.save(f'{filename}.npy', meta)
        print ('file Meta Saved')
 
        tweets_dt.to_csv(file_name, index=False,compression='bz2')
        print(f'{len(tweets_dt)} Tweets found and saved')
        
    else:
        create_dt = time.strftime("%d/%m/%Y %H:%M:%S",time.strptime(time.ctime(os.path.getmtime(file_name))))
        print(f'Reading {file_name}, created at {create_dt}')
        tweets_dt = pd.read_csv(file_name)
        print(f'File with {len(tweets_dt)} Tweets')
        
    return tweets_dt


### Countries

Getting up to 1000 tweets each country

In [67]:
countries = ['Europe', 'Ireland', 'Portugal', 'France', 'German', 'UK']
filename = 'tweets_agri.bz2'

if os.path.exists(filename) == False: 
    for c in countries:

        q = {
        'query': f'agriculture {c} -is:retweet',
        'max_results': 100,
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        }

        fn = f'tweets_{c}'
        t = get_tweet_v1(q, fn, 10)
        t['country'] = c
        if c == countries[0]:
            tweets = t
        else:
            tweets = pd.concat([tweets, t], ignore_index=True)

    tweets.to_csv(filename, index=False,compression='bz2')
    print(f'File with {len(tweets)} saved.')

else:
    create_dt = time.strftime("%d/%m/%Y %H:%M:%S",time.strptime(time.ctime(os.path.getmtime(filename))))
    print(f'Reading {filename}, created at {create_dt}')
    tweets = pd.read_csv(filename)
    print(f'File with {len(tweets)} Tweets')
    

tweets.sample(3)

Reading tweets_agri.bz2, created at 23/12/2022 16:37:52
File with 1027 Tweets


,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country
916,Agricultural apprentices from our Rosewarne ca...,1604838229684920324,en,everyone,1604838229684920324,21294093,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",['1604838229684920324'],2022-12-19T13:57:05.000Z,NaN,NaN,NaN,UK
652,"""Von der Leyen has signed an agreement between...",1604586138496294919,en,everyone,1604586138496294919,1434697221967163393,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1604586138496294919'],2022-12-18T21:15:21.000Z,NaN,"[{'type': 'quoted', 'id': '1604382370664169473'}]",NaN,German
351,Currency 1940 France 50 Francs Banknote P-85b ...,1605664417785733121,en,everyone,1605664417785733121,1456028517930442755,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605664417785733121'],2022-12-21T20:40:03.000Z,NaN,NaN,NaN,France


# Clean Your Text Data

## Worlds extractions

In [6]:
def clean_tweet(x, link, keyword, usernames):
    list_of_lists =[]
    if link == True:
        list_of_links = []
        words = x.split(' ')
        for word in words:
            if re.search('http', word):
                list_of_links.append(re.split("\W+",word.lower()))
        if len(list_of_links) > 0:
            list_of_lists.append(list_of_links[0])
    
    if keyword == True:
        list_of_keywords = []
        words = x.split()
        for word in words:
            if word.startswith('#'):
                list_of_keywords.append(word)
        if len(list_of_keywords) > 0:
            list_of_lists.append(list_of_keywords)
            
    if usernames == True:
        list_of_usernames = []
        words = x.split()
        for word in words:
            if word.startswith('@'):
                list_of_usernames.append(word.lower().replace('@',''))
        if len(list_of_usernames) > 0:
            list_of_lists.append(list_of_usernames)
    
    return  [item for sublist in list_of_lists for item in sublist]

        
# keyword extraction from tweets
def get_keywords(x):
    list_of_keywords = []
    words = x.split()
    for word in words:
        if word.startswith('#'):
            list_of_keywords.append(word)
    return list_of_keywords


In [7]:
tweets = tweets[tweets.lang == 'en']
tweets = tweets.reset_index(drop = True)

In [8]:
list_of_lists = tweets['text'].apply(lambda tweet : clean_tweet(tweet, link = True, keyword = False, usernames = True))
rem_list = [item for sublist in list_of_lists for item in sublist]

tweets['text_c'] = tweets['text'].apply( lambda tweet : ' '.join([word for word in re.split("\W+",tweet) if word.lower() not in rem_list]))

In [9]:
tweets['keywords'] = tweets['text'].apply( lambda tweet : get_keywords(tweet) )

In [10]:
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords
125,Great to see collaboration between Ireland and...,1605624144279765000,en,everyone,1605624144279765000,923680495405293569,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605624144279765000'],2022-12-21T18:00:01.000Z,NaN,NaN,NaN,Ireland,Great to see collaboration between Ireland New...,[]
305,Lyxor Bloomberg Equal-weight Commodity ex-Agri...,1606204264581300225,en,everyone,1606204264581300225,42410755,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1606204264581300225'],2022-12-23T08:25:13.000Z,NaN,NaN,NaN,UK,Lyxor Bloomberg Equal weight Commodity ex UCIT...,[]


In [11]:
n = 1
print('OLD: ', tweets['text'][n], '\n')
print('NEW: ', tweets['text_c'][n])

OLD:  @Nien72521217 @divyanshu3pathi The per Capita story doesn't correlate

When I see 1000 AD, W Europe didn't have trade routes established, didn't have industrial revolution. Had largely constrained agriculture due to nature of their land. No known irrigation projects.

Yet the per Capita is high? 

NEW:  per Capita story doesn correlate When I see 1000 AD W Europe didn have routes established didn have industrial revolution Had largely constrained due to nature of their land No known irrigation projects Yet per Capita is high


## PoterStemmer

In [12]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [13]:
def stremming(df, text_col, name_new_col):
    # Store the column of the dataframe named as "text"
    X = df[text_col]
    cleaned_data=[]
    # For loop from first value to length(X), ^a-zA-Z means include small and capital case letters
    for i in range(len(X)):
        text = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
        text = text.lower().split()
        text = [stemmer.stem(word) for word in text if (word not in stop_words) and (word not in punct)]
        text = ' '.join(text)
        df.loc[ i ,name_new_col] = text
    print('Stremmer done!')

In [14]:
stremming(tweets, 'text_c', 'text_ps')

Stremmer done!


In [15]:
print(tweets['text'][0], '\n')
print(tweets['text_ps'][0], '\n')

The knowledge of how to make pottery spread like wildfire between the hunter-gatherers of Stone Age Europe, covering thousands of miles in a few hundred years, much faster than agriculture https://t.co/Iu2YACvIGN 

knowledg make potteri spread like wildfir hunter gather stone age europ cover thousand mile hundr much faster 



# Sentiment Analyzes

TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more. [link](https://textblob.readthedocs.io/en/dev/index.html)

In [16]:
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps
82,Profresh prevents heating and #feed deteriorat...,1604870236565635075,en,everyone,1604870236565635075,1650441565,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1604870236565635075'],2022-12-19T16:04:16.000Z,NaN,NaN,NaN,Europe,Profresh prevents heating feed deterioration i...,"[#feed, #grains, #animal, #silage, #agricultur...",profresh prevent heat feed deterior store grai...
588,To mark UN International Migrants Day (18th De...,1604402086778634241,en,everyone,1604402086778634241,2242507148,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",['1604402086778634241'],2022-12-18T09:04:00.000Z,NaN,NaN,NaN,UK,To mark UN International Migrants Day 18th Dec...,[#MigrantsDay],mark un intern migrant day th dec partner deve...


In [17]:
for i in tweets.index:
    text = tweets.loc[i,'text_ps']
    tweets.loc[i, 'TextBlob'] = TextBlob(text).sentiment.polarity
    #print(TextBlob(text).sentiment.polarity)
    tweets.loc[i, 'Vader'] = SentimentIntensityAnalyzer().polarity_scores(text)['compound']
    #print(SentimentIntensityAnalyzer().polarity_scores(text)['compound'], '\n')
    
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps,TextBlob,Vader
380,Plan to boost productivity &amp; innovation in...,1605601518371094528,en,everyone,1605601518371094528,1393216141502386176,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",['1605601518371094528'],2022-12-21T16:30:07.000Z,NaN,NaN,NaN,UK,Plan to boost productivity amp innovation in l...,[],plan boost product amp innov launch,0.0,0.4019
269,"🌱 @esa, @DLR_en &amp; the German Federal Offic...",1604431341143171073,en,everyone,1604431341143171073,4918687943,"{'retweet_count': 8, 'reply_count': 0, 'like_c...",['1604431341143171073'],2022-12-18T11:00:15.000Z,NaN,NaN,NaN,German,esa amp German Federal Office of Food BLE orga...,"[#Agrifood, #IGW2023.]",esa amp german feder offic food ble organis sp...,-0.1,0.2960


In [18]:
def get_polarity(df, col):

    polarity = 0
    neutral = 0
    wpositive = 0
    positive = 0
    spositive = 0
    wnegative = 0
    negative = 0
    snegative = 0
     
    for t in df.index:
        
        v = df.loc[t, col]
        polarity += v  # adding up polarities to find the average later

        if (v == 0):  # adding reaction of how people are reacting to find average later
            neutral += 1
            desc = 'neutral'
        elif (v > 0 and v <= 0.3):
            wpositive += 1
            desc ='weak_positive'
        elif (v > 0.3 and v <= 0.6):
            positive += 1
            desc = 'positive'
        elif (v > 0.6 and v <= 1):
            spositive += 1
            desc = 'strong_positive'
        elif (v > -0.3 and v <= 0):
            wnegative += 1
            desc = 'weak_negative'
        elif (v > -0.6 and v <= -0.3):
            negative += 1
            desc = 'negative'
        elif (v > -1 and v <= -0.6):
            snegative += 1
            desc = 'strong_negative'
         
        df.loc[t, f'{col}_desc'] = desc
        

    return {'polarity_sum':polarity,
            'polarity_mean':(polarity / len(df)),
            'neutral':neutral,
            'strong_positive':spositive,
            'positive':positive,
            'weak_positive':wpositive,
            'weak_negative':wnegative,
            'negative':negative,
            'strong_negative':snegative}




In [19]:
get_polarity(tweets,'TextBlob')

{'polarity_sum': 43.62266860916862,
 'polarity_mean': 0.06837408872910442,
 'neutral': 265,
 'strong_positive': 18,
 'positive': 68,
 'weak_positive': 174,
 'weak_negative': 91,
 'negative': 13,
 'strong_negative': 8}

In [20]:
get_polarity(tweets,'Vader')

{'polarity_sum': 109.63159999999992,
 'polarity_mean': 0.1718363636363635,
 'neutral': 209,
 'strong_positive': 99,
 'positive': 154,
 'weak_positive': 70,
 'weak_negative': 39,
 'negative': 42,
 'strong_negative': 25}

In [21]:
tweets.sample()

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
42,@CrocketLawncha @FinanzamtBayern @GreenTextRep...,1605692513083215872,en,everyone,1605723727035092994,1411339294933913602,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",['1605723727035092994'],2022-12-22T00:35:44.000Z,9.096129e+17,"[{'type': 'replied_to', 'id': '160572336195436...",NaN,Europe,bro there were lions every where than 50 giant...,[],bro lion everi giant mammal america none exist...,-0.133333,0.0,weak_negative,neutral


In [22]:
##Checking Twitters

n = 3
print('Text Original:', tweets.loc[n, 'text'], '\n',
     '-------------------------------------------------------------------------------------------')
print('Text Clear:', tweets.loc[n, 'text_c'], '\n',
     '-------------------------------------------------------------------------------------------')

print('Text Steammed:', tweets.loc[n, 'text_ps'], '\n',
     '-------------------------------------------------------------------------------------------')

print('KeyWords:', tweets.loc[n, 'keywords'], '\n',
     '-------------------------------------------------------------------------------------------')

print('TextBlob: ',tweets.loc[n, 'TextBlob'], tweets.loc[n, 'TextBlob_desc'])
print('Vader: ', tweets.loc[n, 'Vader'], tweets.loc[n, 'Vader_desc'])

Text Original: @RonFilipkowski Ron, you are missing the point.  Putin doesn’t want peace, Putin wants Ukraine’s oil, gas and agriculture (wheat) to have a stranglehold on Europe!!! 
 -------------------------------------------------------------------------------------------
Text Clear: Ron you missing point Putin doesn want peace Putin wants s oil gas wheat to have a stranglehold on Europe 
 -------------------------------------------------------------------------------------------
Text Steammed: ron miss point putin want peac putin want oil ga wheat stranglehold europ 
 -------------------------------------------------------------------------------------------
KeyWords: [] 
 -------------------------------------------------------------------------------------------
TextBlob:  0.0 neutral
Vader:  0.0 neutral


In [23]:
tweets.iloc[:, -6:]

,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
0,[],knowledg make potteri spread like wildfir hunt...,0.200000,0.3612,weak_positive,positive
1,[],per capita stori correl see ad w europ rout es...,0.017500,0.0000,weak_positive,neutral
2,[],everyth need chang villag settl illo tempor hu...,0.000000,0.7717,neutral,strong_positive
3,[],ron miss point putin want peac putin want oil ...,0.000000,0.0000,neutral,neutral
4,[],russia one encourag us aid amp assist help peo...,0.000000,0.4019,neutral,positive
...,...,...,...,...,...,...
633,[],ye reform set safe amp legal rout process refu...,0.350000,0.8689,positive,strong_positive
634,"[#BiggestJobonEarth, #agriculture]",case miss catch episod biggestjobonearth podca...,-0.500000,0.1531,negative,weak_positive
635,[],respons contribut transform new strategi,0.136364,0.0000,weak_positive,neutral
636,[],eu minist call rethink free,0.400000,0.5106,positive,positive


## Sarcasm

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
news = pd.read_csv('sarcasm_headlines.bz2')
news.sample()

,headline,is_sarcastic
13446,hillary clinton wows russians with poignant ch...,1


In [26]:
print(news.isnull().any(axis = 0))

headline        False
is_sarcastic    False
dtype: bool


In [27]:
stremming(news, 'headline', 'headline_ps')

Stremmer done!


In [28]:
news.sample(2)

,headline,is_sarcastic,headline_ps
13845,woman probably just made up rape story in orde...,1,woman probabl made rape stori order get threat...
12416,25-pound ham wedged in parents' refrigerator,1,pound ham wedg parent refriger


In [29]:
def create_features(df, col, max_feat):
    features = df[col]

    # vectorizing the data with maximum features
    tv = TfidfVectorizer(max_features = max_feat)
    features = list(features)
    features = tv.fit_transform(features).toarray()
    
    return features

In [54]:
features = create_features(news, 'headline_ps', 3300)
labels = news['is_sarcastic']

In [55]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

In [56]:
print('\nLinear Support Vector Classifier:')
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print('Train: ',lsvc.score(features_train, labels_train))
print('Test: ',lsvc.score(features_test, labels_test))


print('\nGaussuan Naive Bayes:')
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('Train: ',gnb.score(features_train, labels_train))
print('Test: ',gnb.score(features_test, labels_test))


print('\nLogistic Regression:')
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('Train: ',lr.score(features_train, labels_train))
print('Test: ',lr.score(features_test, labels_test))


print('\nRandom Forest Classifier:')
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('Train: ',rfc.score(features_train, labels_train))
print('Test: ',rfc.score(features_test, labels_test))


Linear Support Vector Classifier:
Train:  0.8462933039057265
Test:  0.7784431137724551

Gaussuan Naive Bayes:
Train:  0.750679856540417
Test:  0.7215568862275449

Logistic Regression:
Train:  0.8314349899499468
Test:  0.782185628742515

Random Forest Classifier:
Train:  0.9827375556694123
Test:  0.7365269461077845


### Prev

In [57]:
prev = create_features(tweets, 'text_ps', 3300)

In [58]:
tweets['is_sarcastic_lsvc'] = lsvc.predict(prev)

In [59]:
tweets['is_sarcastic_lr'] = lr.predict(prev)

In [60]:
tweets.sample(4)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,...,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc,is_sarcastic,is_sarcastic_lsvc,is_sarcastic_lr
296,Defining a clear vision: how regulation impact...,1606238961076092928,en,everyone,1606238961076092928,1453665282484998149,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1606238961076092928'],2022-12-23T10:43:05.000Z,NaN,...,Defining a clear vision how regulation impacts...,[],defin clear vision regul impact innov,0.100000,0.5574,weak_positive,positive,0,0,0
315,@SammyMarkleReal The Industrial revolution bro...,1605768828016726016,en,everyone,1606099866135932928,1391360513834209282,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",['1606099866135932928'],2022-12-23T01:30:22.000Z,1.585831e+18,...,Industrial revolution brought everyone into ci...,[],industri revolut brought everyon citi back sus...,0.000000,-0.5423,neutral,negative,1,0,0
434,@ellegist @danwaterfield According to the UK g...,1604919853084598272,en,everyone,1605247678845259777,1159804461839212544,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",['1605247678845259777'],2022-12-20T17:04:05.000Z,1.065047e+08,...,According to government statistics land use ha...,[],accord govern statist land use quit stabl sinc...,-0.022222,0.0000,weak_negative,neutral,0,0,0
15,Organic farming would be a solution to fight g...,1606244055033122817,en,everyone,1606244055033122817,1097880624176816128,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1606244055033122817'],2022-12-23T11:03:19.000Z,NaN,...,Organic would be a solution to fight global wa...,[#IMCAP],organ would solut fight global warm possibl tr...,0.184091,-0.1779,weak_positive,weak_negative,1,0,0


In [64]:
##Checking Twitters

n = 90
print('Text Original:', tweets.loc[n, 'text'], '\n',
     '-------------------------------------------------------------------------------------------')
print('Text Clear:', tweets.loc[n, 'text_c'], '\n',
     '-------------------------------------------------------------------------------------------')

print('Text Steammed:', tweets.loc[n, 'text_ps'], '\n',
     '-------------------------------------------------------------------------------------------')

print('KeyWords:', tweets.loc[n, 'keywords'], '\n',
     '-------------------------------------------------------------------------------------------')

print('TextBlob: ',tweets.loc[n, 'TextBlob'], tweets.loc[n, 'TextBlob_desc'])
print('Vader: ', tweets.loc[n, 'Vader'], tweets.loc[n, 'Vader_desc'])
print('Sarcasm (lsvc): ', tweets.loc[n, 'is_sarcastic_lsvc'])
print('Sarcasm (lr): ', tweets.loc[n, 'is_sarcastic_lr'])


Text Original: @Celtic_Films Really I think a lot of it comes down to agriculture spreading from the middle east into europe 
 -------------------------------------------------------------------------------------------
Text Clear: Really I think a lot of it comes down to spreading from middle east into europe 
 -------------------------------------------------------------------------------------------
Text Steammed: realli think lot come spread middl east europ 
 -------------------------------------------------------------------------------------------
KeyWords: [] 
 -------------------------------------------------------------------------------------------
TextBlob:  0.0 neutral
Vader:  0.0 neutral
Sarcasm (lsvc):  0
Sarcasm (lr):  0
